In [1]:
# import dependencies
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [2]:
from google.colab import files
uploaded = files.upload()

Saving mushrooms.csv to mushrooms.csv


In [3]:
import io

df = pd.read_csv(io.BytesIO(uploaded['mushrooms.csv']))

In [4]:
# check dataframe
df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [5]:
# get column types, counts and column names
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   class                     8124 non-null   object
 1   cap-shape                 8124 non-null   object
 2   cap-surface               8124 non-null   object
 3   cap-color                 8124 non-null   object
 4   bruises                   8124 non-null   object
 5   odor                      8124 non-null   object
 6   gill-attachment           8124 non-null   object
 7   gill-spacing              8124 non-null   object
 8   gill-size                 8124 non-null   object
 9   gill-color                8124 non-null   object
 10  stalk-shape               8124 non-null   object
 11  stalk-root                8124 non-null   object
 12  stalk-surface-above-ring  8124 non-null   object
 13  stalk-surface-below-ring  8124 non-null   object
 14  stalk-color-above-ring  

In [12]:
# drop columns from VIF analysis
new_df = df[['class','cap-shape','cap-surface','cap-color','bruises','odor','gill-spacing','habitat',
      'gill-size','gill-color','stalk-shape','stalk-root']]

In [13]:
# get value counts for y
new_df['class'].value_counts()

e    4208
p    3916
Name: class, dtype: int64

In [14]:
# Convert categorical data to numeric with `pd.get_dummies`
new_df = pd.get_dummies(new_df)

new_df.head()

,class_e,class_p,cap-shape_b,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_f,cap-surface_g,...,gill-color_u,gill-color_w,gill-color_y,stalk-shape_e,stalk-shape_t,stalk-root_?,stalk-root_b,stalk-root_c,stalk-root_e,stalk-root_r
0,0,1,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0
1,1,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,1,0,0
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
3,0,1,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0
4,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,1,0


In [16]:
# split our data into our features and target arrays
X = new_df.drop(columns = ['class_e', 'class_p'], axis =1)
y = new_df[['class_e']]

In [17]:
# split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=23)

In [18]:
# create a StandardScaler instances
scaler = StandardScaler()

# fit the StandardScaler
X_scaler = scaler.fit(X_train)

# scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=6, activation="relu", input_dim=61))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=4, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 6)                 372       
                                                                 
 dense_4 (Dense)             (None, 4)                 28        
                                                                 
 dense_5 (Dense)             (None, 1)                 5         
                                                                 
Total params: 405
Trainable params: 405
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [24]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
191/191 [==============================] - 3s 4ms/step - loss: 0.3567 - accuracy: 0.8602
Epoch 2/50
191/191 [==============================] - 1s 3ms/step - loss: 0.1130 - accuracy: 0.9688
Epoch 3/50
191/191 [==============================] - 1s 3ms/step - loss: 0.0575 - accuracy: 0.9888
Epoch 4/50
191/191 [==============================] - 1s 3ms/step - loss: 0.0355 - accuracy: 0.9934
Epoch 5/50
191/191 [==============================] - 1s 3ms/step - loss: 0.0250 - accuracy: 0.9952
Epoch 6/50
191/191 [==============================] - 0s 2ms/step - loss: 0.0176 - accuracy: 0.9966
Epoch 7/50
191/191 [==============================] - 0s 3ms/step - loss: 0.0134 - accuracy: 0.9969
Epoch 8/50
191/191 [==============================] - 0s 2ms/step - loss: 0.0107 - accuracy: 0.9975
Epoch 9/50
191/191 [==============================] - 0s 2ms/step - loss: 0.0085 - accuracy: 0.9984
Epoch 10/50
191/191 [==============================] - 0s 3ms/step - loss: 0.0068 - accuracy: 0.9990

In [25]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

64/64 - 0s - loss: 8.3531e-05 - accuracy: 1.0000 - 297ms/epoch - 5ms/step
Loss: 8.353116572834551e-05, Accuracy: 1.0
